# Домашнее задание №2

**

## Часть 1. ML workflow (**всего 5 баллов**)

In [67]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы. 

In [108]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [129]:
y = df['quality']
X = df.drop(['quality'], axis=1)

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [110]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [119]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train,y_train) 
pred_train = lr.predict(X_train)
pred_test = lr.predict(X_test)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [120]:
from sklearn.metrics import mean_squared_error, r2_score
print("Train MSE: {}".format(mean_squared_error(y_train, pred_train)))
print("Test MSE: {}".format(mean_squared_error(y_test, pred_test)))
print("Train R^2: {}".format(r2_score(y_train, pred_train)))
print("Train R^2: {}".format(r2_score(y_test, pred_test)))

Train MSE: 0.40116626958113233
Test MSE: 0.4592749592402099
Train R^2: 0.3768316904669994
Train R^2: 0.3139283266620997


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [113]:
from sklearn.model_selection import cross_validate

cv_res = cross_validate(lr,
                     X,
                     y,
                     scoring='r2',
                     cv=5
                    )

print(f"test R^2 errors are {cv_res['test_score']}")
print(f"mean test r^2 = {cv_res['test_score'].mean()}")

test R^2 errors are [0.13200871 0.31858135 0.34955348 0.369145   0.2809196 ]
mean test r^2 = 0.2900416288421966


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [114]:
from sklearn.linear_model import Lasso

for a in np.arange(0.1, 1.1, 0.1): 
    if a == 0:
        a += 0.00000001
    lasso = Lasso(alpha=a)
    lasso.fit(X_train, y_train)

    y_pred_train = lasso.predict(X_train) 
    y_pred_test = lasso.predict(X_test)

    print('alpha={}'.format(a))
    print('Train MSE:', mean_squared_error(y_train, y_pred_train))
    print('Test MSE:', mean_squared_error(y_test, y_pred_test))
    print(lasso.coef_,'\n')

alpha=0.1
Train MSE: 0.490045291386179
Test MSE: 0.5084679025681912
[ 0.03343109 -0.          0.          0.         -0.          0.00863865
 -0.00417293 -0.         -0.          0.          0.25433892] 

alpha=0.2
Train MSE: 0.5303974794732216
Test MSE: 0.5484428682941581
[ 0.         -0.          0.          0.         -0.          0.00695934
 -0.00463962 -0.         -0.          0.          0.15282863] 

alpha=0.30000000000000004
Train MSE: 0.5795520645881872
Test MSE: 0.603531424131992
[ 0.         -0.          0.          0.         -0.          0.00591201
 -0.00503797 -0.         -0.          0.          0.05516951] 

alpha=0.4
Train MSE: 0.6168490851951058
Test MSE: 0.6456471236445623
[ 0.         -0.          0.          0.         -0.          0.00444396
 -0.0050318  -0.         -0.          0.          0.        ] 

alpha=0.5
Train MSE: 0.6191401061143759
Test MSE: 0.6461541076381413
[ 0.         -0.          0.          0.         -0.          0.00242982
 -0.00450042 -0.    

### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [115]:
from sklearn.model_selection import GridSearchCV

n_alphas = 11
alphas = np.linspace(0.1, 1.1, n_alphas)

params = {'alpha':alphas}
cv = GridSearchCV(lasso,
                  params,
                  scoring='neg_mean_squared_error',
                  cv=5
                 )
cv.fit(X, y)

print(cv.best_estimator_)
print(cv.best_score_)

Lasso(alpha=0.1)
-0.5087155341262227


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [116]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_pred_test_2 = lasso.predict(X_test)
print('Test R^2:', r2_score(y_test, y_pred_test_2))


Test R^2: 0.24044318608034898


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [132]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

y = df['quality']
X = df.drop(['quality'], axis=1)

scaler = StandardScaler()
scaler.fit_transform(X.values.reshape(-1,1))

pipi = Pipeline([('PolynomialFeatures', PolynomialFeatures(degree=2)),
                ('model_', LinearRegression())])
cv_pipi = cross_validate(pipi,
                        X,
                        y,
                        scoring='r2',
                        cv=5)
print(cv_pipi['test_score'].mean())

0.23009617062168103


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [133]:
pipi.fit(X_train,y_train) 
pipi_pred_train = pipi.predict(X_train)
pipi_pred_test = pipi.predict(X_test)
print("Train MSE: {}".format(mean_squared_error(y_train, pipi_pred_train)))
print("Test MSE: {}".format(mean_squared_error(y_test, pipi_pred_test)))
print("Train R^2: {}".format(r2_score(y_train, pipi_pred_train)))
print("Test R^2: {}".format(r2_score(y_test, pipi_pred_test)))

Train MSE: 0.3487035464678929
Test MSE: 0.48841949776857624
Train R^2: 0.4583268433623586
Test R^2: 0.2703917873526285


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

1) Модель показала не очень хорощее качество, что мы видим из значения r2_score 0,37 и 0,33 на train и test соответственно
2) Регуляризация не улучшила модель. r2_score вышел ниже, чем в стандартной модели
3) Добавление полиномов не улучшило качество модели, что показал r2_score на тесте. Очевидна переобученность, т.к. на train r2_score увеличился заметно. Различие значений на train и test как раз и являются показателем переобученности.

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [127]:
y = df['quality']
X = df.drop(['quality'], axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X.values.reshape(-1,1))
y = scaler.fit_transform(y.values.reshape(-1,1))
pipipi = Pipeline([('PolynomialFeatures', PolynomialFeatures(degree=2)),
                ('model_', LinearRegression())])
cv_pipipi = cross_validate(pipi,
                        X,
                        y,
                        scoring='r2',
                        cv=5)
pipipi.fit(X_train,y_train) 
pipipi_pred_train = pipi.predict(X_train)
pipipi_pred_test = pipi.predict(X_test)
print("Train MSE: {}".format(mean_squared_error(y_train, pipipi_pred_train)))
print("Test MSE: {}".format(mean_squared_error(y_test, pipipi_pred_test)))
print("Train R^2: {}".format(r2_score(y_train, pipipi_pred_train)))
print("Test R^2: {}".format(r2_score(y_test, pipipi_pred_test)))

ValueError: Found input variables with inconsistent numbers of samples: [17589, 1599]

## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [134]:
sales = pd.read_csv('sales_train_v2.csv')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [135]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [136]:
all_data

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0
...,...,...,...,...
10768834,59,22162,33,0.0
10769024,59,22163,33,0.0
10769690,59,22164,33,0.0
10771216,59,22166,33,0.0


### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [137]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862128


#### Способ 2

In [138]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862128


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [139]:
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder

In [140]:
from sklearn.model_selection import KFold

item_id_target_mean = all_data.groupby('item_id').target.mean()
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

kf = KFold(n_splits=5, shuffle=False)
for train_index, test_index in kf.split(all_data):
    X_train = all_data.iloc[train_index]
    X_test = all_data.iloc[test_index]
    item_id_target_mean = X_train.groupby('item_id').target.mean()
    X_test['item_target_enc'] = X_test['item_id'].map(item_id_target_mean)
    all_data.iloc[test_index] = X_test
      
all_data['item_target_enc'].fillna(0.3343, inplace=True)
encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(round(corr, 4))

<ipython-input-140-c0ee272db2a2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['item_target_enc'] = X_test['item_id'].map(item_id_target_mean)


0.4165


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [141]:
sum_tg = all_data.groupby('item_id')['target'].transform('sum')
n_objects = all_data.groupby('item_id')['target'].transform('count')
all_data['item_target_enc'] = (( sum_tg - all_data['target']) / (n_objects - 1))
all_data['item_target_enc'].fillna(0.3343, inplace=True)
encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(round(corr, 4))

0.4804


In [142]:
a = round(corr, 4) - 0.0001
print(a, ':)')

0.4803 :)


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [143]:
mean_target = all_data.groupby('item_id')['target'].transform('mean')
n_rows = all_data.groupby('item_id')['target'].transform('count')
alpha = 100
globalmean = 0.3343
all_data['item_target_enc'] = ((mean_target * n_rows) + (globalmean * alpha)) / (n_rows +alpha)
all_data['item_target_enc'].fillna(0.3343, inplace=True)

encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(round(corr, 4))

0.4818


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [144]:
summa = all_data.groupby('item_id')['target'].cumsum() - all_data['target']

count_ = all_data.groupby('item_id').cumcount()

all_data['item_target_enc'] = summa / count_
all_data['item_target_enc'].fillna(0.3343, inplace=True)

encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(round(corr, 4))

0.5025


Ожидаемый ответ 0.5025